## Abstract

This project focuses on the development and implementation of a predictive model for crop selection based on a comprehensive agricultural dataset. The dataset comprises multiple features such as nutrients in soil, weather conditions, and other agronomic factors, with the target variable being the crop type.

The primary objective was to analyze the dataset to understand the key factors influencing crop selection, followed by the construction of a predictive model to recommend the most suitable crops for given conditions. Various data preprocessing techniques and exploratory data analysis (EDA) were employed to enhance the quality of the data.

Multiple machine learning algorithms were evaluated, including decision trees, random forests, and logistic regression, to determine the most accurate model. The final model was selected based on its accuracy.

Additionally, a Streamlit application was developed to provide an interactive user interface for stakeholders, enabling them to input specific agronomic conditions and receive crop recommendations with associated probabilities. This tool aims to support farmers and agricultural planners in making informed decisions to optimize crop yields and sustainability. Additionally, it ensures the validity of the model presented.

## Introduction

We have a dataset that showcases what crop is best suited for a combination of different features(quantitatively) like Nitrogen, Phosphorus, Potassium and so on.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec, GridSpecFromSubplotSpec
import seaborn as sb
import numpy as np
import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer 
from sklearn.discriminant_analysis import StandardScaler 
from sklearn.preprocessing import QuantileTransformer 
from scipy.stats import boxcox, yeojohnson
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [ ]:
file_path=r"C:\Users\subha\Desktop\AgriSage\Data\Crop_Recommendation.csv"
raw_data= pd.read_csv (file_path)
raw_data.info()
target = 'Crop'
features= raw_data.columns[:-1]
a1 = raw_data.copy()

In [ ]:
import warnings

# Suppressing specific warning
warnings.filterwarnings("ignore", category=FutureWarning)

# Suppressing all warnings
warnings.filterwarnings("ignore")

## Exploratory Data Analysis (EDA)


In [ ]:
def plot_counts(data, features, hue=None):
    n_cols = 2 
    n_rows = int(np.ceil(len(features)/n_cols)) 

    # Create figure
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 4*n_rows))
    # Flatten the axes array for easier indexing
    axes = axes.flatten()

    for i, feature in enumerate(features):
        #bar_container = axes[i].bar(df[feature].value_counts().index, df[feature].value_counts().values)
        sb.countplot(
        data = data,
        x = feature,
        palette = 'Paired',
        hue = hue,
        ax = axes[i]
        )
        for p in axes[i].patches:
            label = p.get_height()
            x = p.get_x() + p.get_width() / 2 # Width of the plot
            y = p.get_height() # Height of the plot
            
            rotation = 0
            if hue is not None:
                rotation = 30  # Rotate annotations by 30 degrees if hue is not None

    axes[i].annotate(
        '{:.0f}'.format(label),
        (x, y),
        ha = 'center',
        va = 'center',
        size = 12,
        xytext = (0, 5),
        textcoords = 'offset points',
        rotation = rotation
    )
    axes[i].set(ylabel='Count', title=feature, xlabel='')
    axes[i].tick_params(axis='x', rotation=30)

    # If the number of features is odd, remove the empty subplot
    if len(features) % 2 != 0:
        fig.delaxes(axes[-1])

plt.tight_layout()
plt.show()
plot_counts(raw_data, [target])

The above figure helps us in a clear visualization of our data. 

1. Quantification: We have 22 possible targets that is crops for our datset and since each crop has a 100 samples, we have 2200 samples.

2. Class Balance: The fact that each crop has an equal number of samples (100) suggests that the dataset is balanced. This is beneficial for training machine learning models as it can prevent bias towards any particular class.

3. Granularity: In a classification problem like the one presented here, having 100 samples per target allows for a detailed representation of each class.

However, the number of samples is a subject to change after further EDA.


In [ ]:
plt.figure(figsize=(12, 10))
sb.heatmap(raw_data.corr(numeric_only=True), annot=True, fmt='.2f')
plt.show()

The above figures tells us that Phosphorus and Potassium have a high positive correation. This can be due to several underlying reasons related to soil chemistry, plant physiology, and farming practices. Here are some common reasons why these two nutrients might show correlation:

1. Soil Composition: Phosphorus (P) and potassium (K) availability in soil can be influenced by similar factors such as soil type, pH levels, and organic matter content.

2. Fertilizer Application: Farmers often apply fertilizers that contain both phosphorus and potassium together. This simultaneous application can lead to their concentrations being correlated in the soil.

3. Sampling and Analysis: Sometimes, the correlation observed could be due to the way samples are collected or analyzed. If samples are taken from similar locations or depths within a field, they may show similar nutrient profiles.

4. Environmental Factors: Environmental conditions such as rainfall, temperature, and humidity can affect the mobility and availability of both phosphorus and potassium in the soil. Similar environmental impacts can result in correlated values. In our datset since these factors would be same for a particular area, the correlation is understandable.

Highly correlated variables can adversely affect the performance of predictive models, particularly in regression and classification tasks:
 
1. It can lead to multicollinearity, where the coefficients become unstable and difficult to interpret in regression models.

2. It can also lead to OVERFITTING occurs when the model captures noise or random fluctuations rather than underlying patterns.


In [ ]:
def dist_plot(data, feature_list):
    n_cols= 2 
    n_rows = int(np.ceil(len(feature_list)/n_cols)) 
    # Creating figure
    fig = plt.figure(figsize=(16, 4*n_rows))
    outer = GridSpec(n_rows, n_cols, wspace=0.2, hspace=0.3)

    for i in range(len(feature_list)):
        inner = GridSpecFromSubplotSpec(2, 1, subplot_spec=outer[i], 
                                                 wspace=0.1, hspace=0.1, height_ratios=(0.15, 0.85))
        ax_box= plt.Subplot(fig, inner[0])
        sb.boxplot(data=data, x=feature_list[i], color='lightblue', ax=ax_box)
        ax_box.set_xlabel('')
        fig.add_subplot(ax_box)

        mean_value = data[feature_list[i]].mean()
        median_value = data[feature_list[i]].median()
        ax_hist = plt.Subplot(fig, inner[1])
        sb.histplot(data=data, x=feature_list[i], kde=True, ax=ax_hist)
        ax_hist.axvline(mean_value, color='green', linestyle='dotted', linewidth=2, label='Mean')
        ax_hist.axvline(median_value, color='purple', linestyle='dotted', linewidth=2, label='Median')
        ax_hist.legend(loc='lower right', fontsize=10)

        # Calculate skewness and kurtosis
        skewness = data[feature_list[i]].skew()
        kurt = data[feature_list[i]].kurt()
        if skewness < 0:
            x=0.25
        else:
            x=0.95
        # Add skewness and kurtosis as text on the histogram plot
        ax_hist.text(x, 0.85, f"Skewness: {skewness:.2f}\nKurtosis: {kurt:.2f}", 
                         transform=ax_hist.transAxes, verticalalignment='top', horizontalalignment='right',
                         bbox=dict(facecolor='white', edgecolor='gray', boxstyle='round,pad=0.5'),
                    fontsize=10)
        fig.add_subplot(ax_hist)
    plt.tight_layout()
    plt.show(block=False)
dist_plot(raw_data, features)

From the above graph, we are able to easily identify the outliers that need to be dealt with as they can negatively influence the performance of machine learning models. These are a few outliers:

1. 100 > Potassium> 200

2. 15> Temperature >40

3. 4> pH_Value >8

4. count > 250

By addressing outliers, we can improve the robustness and reliability of your models. It can also help in better visualization.

## Handling Outliers

The Interquartile Range (IQR) method is a common statistical technique used to identify and handle outliers in a dataset. The IQR is the range between the first quartile (Q1) and the third quartile (Q3) of the dataset.


In [ ]:
Q1= a1[features].quantile(0.25)
Q3 = a1[features].quantile(0.75)
IQR = Q3 - Q1 


print('25th percentile of the given data is \n', Q1)
print('75th percentile of the given data is \n', Q3)
print('Interquartile range is\n', IQR)

In [ ]:
lower_bound= Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print('lower bound of the given data is \n', lower_bound)
print('upper bound of the given data is \n', upper_bound)

In [ ]:
outliers = (a1[features] < lower_bound) | (a1[features] > upper_bound)

a1_no_outliers = a1[~outliers.any(axis=1)]

a1_no_outliers.reset_index(inplace=True, drop=True)

# Displaying the cleaned data
a1_no_outliers.info()

These are the number of crop samples left after removing the outliers along with graph:


In [ ]:
a1_no_outliers[target].value_counts()
plot_counts(a1_no_outliers, [target])

It is clear to us that we cannot remove outliers as it leads to serious reduction of samples for some crops like rice and papaya. Reducing the number of samples can decrease the statistical power of our analysis. With fewer samples, the ability to detect true patterns or relationships in the data diminishes.

Retaining outliers might provide a more comprehensive understanding of the factors influencing crop yield, leading to more robust and reliable predictions. 

We transform the data:


In [ ]:
def transformation_method(data, feature):
    fig, axes = plt.subplots(2, 3, figsize=(12, 9))  # Adjusted for an additional plot

    # Flatten the axes array for easier indexing
    axes = axes.flatten()

    # Hide unused subplot axes
    for ax in axes[6:]:
        ax.axis('off')
        
    # Calculate skewness
    # Original Distribution of the feature
    sb.histplot(data[feature], kde=True, bins=30, palette='viridis', ax=axes[0])
    axes[0].set_title(f'Original {feature} Distribution \n (Skew: {data[feature].skew():.5f})', fontsize=10)
    axes[0].set_xlabel(feature, fontsize=10)
    axes[0].set_ylabel('Frequency', fontsize=10)

    # Log Transformed feature
    data['Log_'+feature] = np.log(data[feature])
    sb.histplot(data['Log_'+feature], kde=True, bins=30, color='orange', ax=axes[1])
    axes[1].set_title(f"Log Transformed {feature} \n (Skew: {data['Log_'+feature].skew():.5f})", fontsize=10)
    axes[1].set_xlabel(f'Log of {feature}', fontsize=10)
    axes[1].set_ylabel('Frequency', fontsize=10)

    # Square Root Transformed feature
    data['Sqrt_'+feature] = np.sqrt(data[feature])
    sb.histplot(data['Sqrt_'+feature], kde=True, bins=30, color='blue', ax=axes[2])
    axes[2].set_title(f"Square Root Transformed {feature} \n (Skew: {data['Sqrt_'+feature].skew():.5f})", fontsize=10)
    axes[2].set_xlabel(f'Square Root of {feature}', fontsize=10)
    axes[2].set_ylabel('Frequency', fontsize=10)
    
    # Box-Cox Transformed feature
    # Applying Box-Cox Transformation after checking all values are positive
    if (data[feature] <= 0).any():
        # Shift the values to make them positive
        shift_value = abs(data[feature].min()) + 1
        data[feature] += shift_value
        print(f"Values of {feature} were shifted to make them positive.")
    data['BoxCox_'+feature], _ = boxcox(data[feature])
    sb.histplot(data['BoxCox_'+feature], kde=True, bins=30, color='red', ax=axes[3])
    axes[3].set_title(f"Box-Cox Transformed {feature} \n Skew: {data['BoxCox_'+feature].skew():.5f})", fontsize=10)
    axes[3].set_xlabel(f'Box-Cox of {feature}', fontsize=10)
    axes[3].set_ylabel('Frequency', fontsize=10)

    # Yeo-Johnson Transformed feature
    # Applying Yeo-Johnson Transformation
    data['YeoJohnson_'+feature], _ = yeojohnson(data[feature])
    sb.histplot(data['YeoJohnson_'+feature], kde=True, bins=30, color='purple', ax=axes[4])
    axes[4].set_title(f"Yeo-Johnson Transformed {feature} \n (Skew: {data['YeoJohnson_'+feature].skew():.5f})", fontsize=10)
    axes[4].set_xlabel(f'Yeo-Johnson of {feature}', fontsize=10)
    axes[4].set_ylabel('Frequency', fontsize=10)

    # Quantile Transformed feature (Normal Distribution)
    # Applying Quantile Transformation to follow a normal distribution
    quantile_transformer = QuantileTransformer(output_distribution='normal', random_state=0)
    data['Quantile_'+feature] = quantile_transformer.fit_transform(data[feature].values.reshape(-1, 1)).flatten()
    sb.histplot(data['Quantile_'+feature], kde=True, bins=30, color='green', ax=axes[5])
    axes[5].set_title(f"Quantile Transformed {feature} (Normal Distn, \n Skew: {data['Quantile_'+feature].skew():.5f})", fontsize=10)
    axes[5].set_xlabel(f'Quantile Transformed {feature}', fontsize=10)
    axes[5].set_ylabel('Frequency', fontsize=10)


    plt.tight_layout(pad=3.0)
    plt.show()

In [ ]:
transformation_method(a1, features[0])

In [ ]:
transformation_method(a1, features[1])

In [ ]:
transformation_method(a1, features[2])

In [ ]:
transformation_method(a1, features[3])

In [ ]:
transformation_method(a1, features[5])

It is evident that the quantile transformation graph looks the best so we'll use quantile transformation on the needed data.

However in some of the graphs above, we see that some transformations result in double peaks. This might indicate that crops might be devided in two groups for each feature. To understand this assumption of ours, we can graphs that show crops vs features, i.e. how different crops behave to different values of features.


In [ ]:
n_cols = 2
n_rows = 4
fig, ax = plt.subplots(n_rows, n_cols, figsize=(16, 4*n_rows))
ax = ax.flatten()

for i, feature in enumerate(features):
    sb.boxplot(data=a1, x=target, y=feature,ax=ax[i] )
    ax[i].tick_params(axis='x', rotation=30)

plt.tight_layout()
plt.show(block=False)

From the above box plots it is now clear to us that crops are be divided into to groups for certain features. 

For example, the Box-Cox transformation and Yeo-Johnson transformation for Nitrogen shows two peaks. This is further confirmed from the box plot above where almost half of the crops reuire low nitrogen and the other half requires high nitrogen. 

This is also the case for certain other features like Potassium and Phosphorus. 

## Label Encoding


In [ ]:
def transform_data_le(df, target, num_features):
    # Ensure num_features is a list
    if isinstance(num_features, pd.Index):
        num_features = num_features.tolist()

    # Encoding target
    lbl_encoder = LabelEncoder()
    df[target+'_Encoded'] = lbl_encoder.fit_transform(df[target])
    
    # Assigning features and labels
    x = df.drop([target, target+'_Encoded'], axis=1)
    y = df[target+'_Encoded']
    
    # Splitting the dataset into train and test sets
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=101)
    
    # Accessing the encoded classes
    encoded_classes = lbl_encoder.classes_
    # Printing the mapping (index corresponds to encoded value, value is the original label)
    for i, label in enumerate(encoded_classes):
        print(f"Encoded Value: {i}, Original Label: {label}")    
    
    # Standardization and Encoding
    # Define transformers for different column types
    std_scaler = StandardScaler()
    quantile_transformer = QuantileTransformer(output_distribution='normal', random_state=0)

    # Combine transformers for specific columns
    preprocessor = ColumnTransformer([
        ("num", std_scaler, num_features),
        ("num_trns", quantile_transformer, num_features)
    ])
     # Fit transformers on training data only
    preprocessor.fit(x_train)

    # Transform train and test data using fitted transformers
    x_train_transformed = preprocessor.transform(x_train)
    x_test_transformed = preprocessor.transform(x_test)

    # Save the test set to CSV files
    x_test.to_csv('x_test.csv', index=False)
    y_test.to_csv('y_test.csv', index=False)
    
    return x_train_transformed, x_test_transformed, y_train, y_test, preprocessor, lbl_encoder
x_train, x_test, y_train, y_test, preprocessor, lbl_encoder = transform_data_le(raw_data, target, features)

## Model comparison


In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import SGDClassifier,  LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier


def model_comparison(x, y, models):
    names = []
    scoring = ['accuracy']
    
    # Create a dataframe to store the different metric values for each algorithm
    df_results = pd.DataFrame(columns=['Algorithm', 'Acc Mean', 'Acc STD'])
    results_acc = [] # List of accuracy scores for each fold of each algorithm
    
    for name, model in models:
        names.append(name)
        kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=101)
        result = cross_validate(model, x, y, cv=kfold, scoring=scoring)
        # Mean and standard deviation of Accuracy scores for the algorithm
        acc_mean = result['test_accuracy'].mean()
        acc_std = result['test_accuracy'].std()
        
        # Create the row of the results
        df_result_row = {'Algorithm': name, 'Acc Mean': acc_mean, 'Acc STD': acc_std}
        # Add the row to the results data frame
        df_results = pd.concat([df_results, pd.DataFrame([df_result_row])], ignore_index=True)
        
        results_acc.append(result['test_accuracy'])

    df_results = df_results.set_index('Algorithm')
    pd.set_option('display.float_format', lambda x: '%.3f' % x)
    # Display the mean and standard deviation of all metrics for all algorithms
    print(df_results) 
    # Convert the results_acc dictionary into a DataFrame for plotting
    results_acc_df = pd.DataFrame(results_acc).T
    print(f"results_acc_df: {results_acc_df}")
    # Create a box plot for the accuracy results
    plt.figure(figsize=(12, 8))
    ax = sb.boxplot(data=results_acc_df)
    ax.set_title('Model Comparison - Accuracy Scores')
    ax.set_xlabel('Algorithm')
    ax.set_ylabel('Accuracy')
    ax.set_xticklabels(names) 
    plt.show()

models = []
models.append(('RFC', RandomForestClassifier()))
models.append(('ABC', AdaBoostClassifier()))
models.append(('GBC', GradientBoostingClassifier()))
models.append(('LR', LogisticRegression())) 
models.append(('SVC', SVC()))
models.append(('DTC', DecisionTreeClassifier()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('GNB', GaussianNB()))
models.append(('XGB', SGDClassifier()))

model_comparison(x_train, y_train, models)

We can clearly discard Ada Boost Classifier.


In [ ]:
selected_models = []
selected_models.append(('RFC', RandomForestClassifier()))
selected_models.append(('GBC', GradientBoostingClassifier()))
selected_models.append(('LR', LogisticRegression())) 
selected_models.append(('SVC', SVC()))
selected_models.append(('DTC', DecisionTreeClassifier()))
selected_models.append(('KNN', KNeighborsClassifier()))
selected_models.append(('GNB', GaussianNB()))
selected_models.append(('XGB', SGDClassifier()))

model_comparison(x_train, y_train, selected_models)

After analyzing the above output, we can finalize Random Forets classifier model because of high accuracy and large size of our dataset.

## Finalizing model


In [ ]:
model = RandomForestClassifier()
model.fit(x_train, y_train)

In [ ]:
y_hat = model.predict(x_test)
print(classification_report(y_test, y_hat))

In [ ]:
def plot_confusion_matrix(y_test, predictions):
    # Plot the confusion matrix
    cf_matrix = confusion_matrix(y_test, predictions)
    fig = plt.subplots(figsize=(10, 8))
    sb.set(font_scale=1.4)
    sb.heatmap(cf_matrix, annot=True, fmt='d')
    plt.xlabel('Predicted Label', fontsize=12)
    plt.xticks(fontsize=12)
    plt.ylabel('True Label', fontsize=12)
    plt.yticks(fontsize=12)
    plt.show()
    
    # Reset font scale to default
    sb.set(font_scale=1)
plot_confusion_matrix(y_test, y_hat)

## Conclusion

The primary objective of this analysis was to develop a robust predictive model for crop prediction. The Random Forest model achieved an accuracy of 99.4%. This results indicate that the model is effective in predicting crop yields. The model's performance was validated using cross-validation techniques, yielding consistent results across different folds.

Compared to other models such as Logistic Regression and Decision Trees, the Random Forest model demonstrated superior performance.

The implementation of this Random Forest model can significantly improve agricultural planning and decision-making, leading to better resource allocation. For instance, the model can help in accurately predicting crop yields, thereby enhancing operational efficiency. In conclusion, the Random Forest model developed in this analysis provides a powerful tool for predicting crop yields, with substantial potential for practical application in agriculture. 

An app has also been developed which takes user input for features and then predicts the top five crops. The app also allows us to validate the model chosen by generating random values from the test dataset. 

## Further links

You can view the project on GitHub for more details.

[View the project on GitHub](https://github.com/subhangi03/AgriSage)